<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## downloading essential modules

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00


In [ ]:
# Using the cs-en translation

In [ ]:
from datasets import load_dataset

import torch
from transformers import MarianMTModel, MarianTokenizer



In [ ]:
ds = load_dataset("wmt/wmt16", "cs-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/177M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/390k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/469k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/997240 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2656 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 997240
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2656
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

In [ ]:
ds['train'][0]

{'translation': {'cs': 'Následný postup na základě usnesení Parlamentu: viz zápis',
  'en': "Action taken on Parliament's resolutions: see Minutes"}}

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-cs-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-cs-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/830k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62509, 512, padding_idx=62508)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62509, 512, padding_idx=62508)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
# https://huggingface.co/docs/transformers/model_doc/marian

In [ ]:
def generate_translation(src_text, max_length = 50):

  # Tokenize input
  encoder_inputs = tokenizer(src_text, return_tensors="pt")

  # intializes the decoder input with decoder start token
  decoder_input = torch.tensor([[model.config.decoder_start_token_id]])

  # change the model to eval mode.
  model.eval()
  with torch.no_grad():

    generated_tokens = []

    while len(generated_tokens) < max_length:

      outputs = model(
          input_ids=encoder_inputs.input_ids,
          attention_mask=encoder_inputs.attention_mask,
          decoder_input_ids=decoder_input
      )


      next_token_logits = outputs.logits[:, -1, :]


      # get the token with maximum logits value
      next_token = torch.argmax(next_token_logits, dim=-1)

      generated_tokens.append(next_token.item())

      if next_token.item() == tokenizer.eos_token_id:
          break
      print(decoder_input)
      decoder_input = torch.cat([decoder_input, next_token.unsqueeze(0)], dim=1)
      print(decoder_input)
    translated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
  return translated_text


In [ ]:

### print("\nStep by step generation:")
# for i in range(len(generated_tokens)):
#     partial_text = tokenizer.decode(generated_tokens[:i+1], skip_special_tokens=True)
#     print(f"Step {i+1}: {partial_text}")

In [ ]:
model.config.num_beams

4

In [ ]:
print(generate_translation(ds['validation'][1]['translation']['cs']))

tensor([[62508]])
tensor([[62508,  1249]])
tensor([[62508,  1249]])
tensor([[62508,  1249,  5365]])
tensor([[62508,  1249,  5365]])
tensor([[62508,  1249,  5365,    28]])
tensor([[62508,  1249,  5365,    28]])
tensor([[62508,  1249,  5365,    28,  1154]])
Anything is possible


In [ ]:
ds["validation"][1]["translation"]["cs"]

'Možné je všechno'

In [ ]:
ds["validation"][1]["translation"]["en"]

'Everything is possible'

{'the': ('yo ', 1.0484259760392978),
 'he ': ('o y', 1.0484259760392978),
 'e n': (' yo', 1.0484259760392978),
 ' na': ('yo ', 1.0484259760392978),
 'nam': ('o y', 1.0484259760392978),
 'ame': (' yo', 1.0484259760392978),
 'hin': ('wha', 1.2930765181575237),
 'ing': ('hat', 1.2930765181575237),
 'ng ': ('at ', 1.2930765181575237),
 'g r': ('t t', 1.2930765181575237),
 ' ra': (' th', 1.2930765181575237),
 'ran': ('the', 1.2930765181575237),
 'an ': ('he ', 1.2930765181575237),
 'n d': ('e h', 1.2930765181575237),
 ' ds': (' he', 1.2930765181575237),
 'dsa': ('hec', 1.2930765181575237),
 'sai': ('eck', 1.2930765181575237)}